<a href="https://colab.research.google.com/github/israa2050/facial-emotion-detection/blob/main/Face%20Emotion%20Detection/Face_Emotion_Detection(WebCam_Video).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Here we open webcam and test the two model Yolov5 for object detection and DL model for Emotion Detection the recorded video link below
https://drive.google.com/file/d/1VZWA4k_omk6yB1_BtVctDfNnknQxo-P_/view?usp=sharing


In [ ]:
# Import Libraries
import cv2
import os
import numpy as np
from keras.models import model_from_json
import tensorflow as tf
import tensorflow_addons as tfa
import torch

In [ ]:
# Yolov5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./best.pt') 
#--------------------------------------------------------------------------------------------------------
# Emotion model 
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
# load weights into new model
emotion_model= tf.keras.models.load_model("./emotionl.h5",custom_objects={'my_custom_loss': tfa.metrics.F1Score})
print("Loaded model from disk")
#--------------------------------------------------------------------------------------------------------
# start the webcam feed
cap = cv2.VideoCapture(0)
#--------------------------------------------------------------------------------------------------------
# pass here your video path
# cap = cv2.VideoCapture("upload your video")
#--------------------------------------------------------------------------------------------------------
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    results = model(frame)
    crops = results.crop(save=True, save_dir='runs/detect/exp')  # specify save dir
#--------------------------------------------------------------------------------------------------------
   # Get the detections as a list of dictionaries
    detections = results.pandas().xyxy[0].to_dict(orient='records')
    # Draw bounding boxes around the detections
    for detection in detections:
        x1, y1, x2, y2 = int(detection['xmin']), int(detection['ymin']), int(detection['xmax']), int(detection['ymax'])
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
#--------------------------------------------------------------------------------------------------------
       # Get coropped images
        gray_img = cv2.cvtColor(crops[detections.index(detection)]["im"], cv2.COLOR_RGB2GRAY)
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(gray_img, (48, 48)), -1), 0)
#--------------------------------------------------------------------------------------------------------
        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x1+5, y1-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
#--------------------------------------------------------------------------------------------------------
    #show Image
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#--------------------------------------------------------------------------------------------------------
cap.release()
cv2.destroyAllWindows()